In [ ]:

"""

Akış:
1) Fine-tune edilmiş teacher model yüklenir
2) Rastgele ve anlamsal(anlamsızda olabilir) promptlar üretilir
3) Teacher model bu promptlara cevaplar üretir
4) (prompt, cevap) çiftleri CSV olarak kaydedilir
"""

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd
import random


"""
Bu bölümde Hugging Face üzerinde yer alan,
önceden davranış verileri üzerine fine-tune edilmiş öğretmen model yüklenir.
Model sadece inference (çıktı üretimi) için kullanılır.

"""
#bylang/teacher1_cosmos_gpt2
#bylang/teacher2_cosmos_gemma
HF_REPO = "--"

teacher_model = AutoModelForCausalLM.from_pretrained(HF_REPO)
teacher_tokenizer = AutoTokenizer.from_pretrained(HF_REPO)

# Eğitim yapılmayacağı için evaluation moduna alınır
teacher_model.eval()


# Rastgele Prompt Üretimi
"""
Teacher modelin çeşitli bağlamlarda cevap üretmesini sağlamak için
günlük senaryolar ve kelime gruplarından oluşan rastgele promptlar oluşturulur.
Bu çeşitlilik, öğrenci modelin genelleme yeteneğini artırmayı hedefler.
"""

kisiler = ["Ali", "Ayşe", "Mehmet", "Elif", "Ahmet"]
yerler = ["park", "okul", "ev", "kafe", "alışveriş merkezi"]
eylemler = ["koşuyor", "çalışıyor", "kitap okuyor", "yemek yapıyor", "çizim yapıyor"]
nesneler = ["top", "kitap", "telefon", "kalem", "çanta"]
duygular = ["mutlu", "üzgün", "heyecanlı", "meraklı", "sakin"]

def generate_random_prompt():
    """
    Rastgele iki prompt türünden birini seçer:
    1) Günlük hayata dayalı kısa bir senaryo
    2) Birden fazla kelimeden oluşan serbest kelime grubu
    """
    prompt_type = random.choice(["daily_scenario", "word_group"])
    
    if prompt_type == "daily_scenario":
        person = random.choice(kisiler)
        place = random.choice(yerler)
        action = random.choice(eylemler)
        feeling = random.choice(duygular)
        return f"{person} {place} de {action}, ve bu sırada {feeling} hissediyor."
    else:
        # Farklı kelime gruplarını bir araya getir
        words = random.sample(
            kisiler + yerler + eylemler + nesneler + duygular, 
            5
        )
        return ", ".join(words)


# Toplam üretilecek prompt sayısı
num_prompts = 200

# Prompt listesi oluşturulur
prompts = [generate_random_prompt() for _ in range(num_prompts)]


#  Teacher Çıktılarının Üretilmesi
"""
Bu aşamada her bir prompt teacher modele verilir
ve modelden olasılıksal (sampling) cevaplar üretilir.
Amaç: Öğrenci modele öğretmen davranışını aktarmaktır.
"""

teacher_outputs = []

for prompt in prompts:
    # Prompt tokenize edilir
    inputs = teacher_tokenizer(prompt, return_tensors="pt")
    
    # Gradient hesaplanmaz (sadece inference)
    with torch.no_grad():
        output_ids = teacher_model.generate(
            **inputs,
            max_length=50,      # Maksimum çıktı uzunluğu
            do_sample=True,     # Rastgele örnekleme aktif
            top_p=0.9,          # Nucleus sampling
            temperature=0.8     # Çeşitlilik kontrolü
        )
    
    # Tokenlar tekrar metne çevrilir
    output_text = teacher_tokenizer.decode(
        output_ids[0],
        skip_special_tokens=True
    )
    
    teacher_outputs.append(output_text)



"""
Oluşturulan (prompt, teacher_cevabı) çiftleri,
öğrenci modelin eğitimi için CSV dosyası olarak kaydedilir.
Bu veri kümesinde gerçek cevaplar yer almaz.
"""

df = pd.DataFrame({
    "input": prompts,
    "target": teacher_outputs
})

df.to_csv("./student_dataset.csv", index=False)

print("Student dataset oluşturuldu ve 'student_dataset.csv' olarak kaydedildi.")
